## Import Required Libraries

In [100]:
import pandas as pd
import numpy as np
from numpy.random import normal
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

## Import data and inspect

In [101]:
df = pd.read_csv('train.csv')

In [84]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [85]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## Processing Age Related Data

In [86]:
df['Age'].isnull().sum()

177

There are 177 null values - we have two choices here: impute the data using the mean/median OR randomly assign a value. To randomly assign a value we will use the Numpy Random function passing in the index/passenger ID as the seed for reproducibility.

In [65]:
df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

#### Impute Age Using Randomly Assigned Value from Mean/Std (Option 1)

In [102]:
seed = np.random.RandomState(15144)
age_estimates = seed.normal(df['Age'].mean(), scale=df['Age'].std(), size=df['Age'].isnull().sum())

In [103]:
age_estimates = [x if x > 0 else df['Age'].mean() for x in age_estimates]
df.loc[df['Age'].isnull(), 'Age'] = age_estimates

In [89]:
df['Age'].describe()

count    891.000000
mean      29.549299
std       14.262134
min        0.420000
25%       20.000000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

#### Imputing Age Using Median/Mean (Option 2)

In [66]:
mean_age_by_gender = df.groupby('Sex').mean()['Age'].to_dict()
null_age = df.loc[df['Age'].isna(), :].Sex.map(mean_age_by_gender)
df.loc[null_age.index, 'Age'] = null_age

## Processing Ticket Related Data

In [ ]:
df['Ticket'] = df['Ticket'].str.replace('\.|/', '')

In [ ]:
temp_ticket = df['Ticket'].str.extract('(\w*)?\s?(.*)')

In [ ]:
for index, row in temp_ticket.iterrows():
    
    if not row[1]:
        
        temp_ticket.iloc[index][1] = row[0]
        temp_ticket.iloc[index][0] = ''

temp_ticket = pd.concat([temp_ticket[1], 
                         pd.get_dummies(temp_ticket[0])[temp_ticket[0].value_counts().keys()[1:10].tolist()]], 
                        axis=1)

In [ ]:
temp_ticket[1] = temp_ticket[1].str.extract('(\d{2,})')
temp_ticket[1] = temp_ticket[1].fillna(0).astype(int)
temp_ticket.rename(columns={1:"TicketID"}, inplace=True)

## Mining Cabin Related Data

#### Extracting Cabin Info into Numerical Values

In [ ]:
cabin_extract = df['Cabin'].str.extract('(\w)(\d*)$')
cabin_extract.head()
cabin_extract = pd.concat([cabin_extract[1], pd.get_dummies(cabin_extract[0])], axis=1).fillna(0)

In [ ]:
cabin_extract.rename(columns={1:'CabinNo'}, inplace=True)
z = dict(zip(cabin_extract.columns[1:].tolist(), [x + '_CABIN' for x in cabin_extract.columns[1:].tolist()]))
cabin_extract.rename(columns=z, inplace=True)
cabin_extract.loc[cabin_extract['CabinNo'] == '', 'CabinNo'] = 0
cabin_extract.loc[:, 'CabinNo'] = cabin_extract['CabinNo'].astype(int, errors='ignore')
cabin_extract.head()

In [ ]:
df = pd.concat([df, cabin_extract], axis=1)

#### Creating Boolean Cabin Value

In [104]:
df.loc[:, 'InCabin'] = ~df['Cabin'].isna()

## Passenger Class Encoded

In [105]:
class_dummies = pd.get_dummies(df['Pclass'])
class_dummies_names = ['class_' + str(x) for x in class_dummies.columns.to_list()]
df[class_dummies_names] = class_dummies

## Embarked Encoded

In [106]:
df = pd.concat([df, pd.get_dummies(df['Embarked'])], axis=1)

## Gender Encoded

In [107]:
df[['female', 'male']] = pd.get_dummies(df['Sex'])

## Feature Engineering

In [ ]:
# TO DO -- 

In [95]:
#df.loc[:, 'Child'] = df['Age'] <= 12

#Life expectancy in 1912 was ~53, well say someone is elderly if they are older than 60
#df.loc[:, 'Elderly'] = df['Age'] >= 60

In [108]:
df['FamilyAboard'] = df['Parch'] + df['SibSp']

In [109]:
df['IsAlone'] = df['FamilyAboard'] == 1

## Dropping unecessary data

In [110]:
df = df.drop(df[df['Embarked'].isna()].index)
drop_columns = ['PassengerId', 'Sex', 'Ticket', 'Cabin', 'Name', 'Pclass', 'Embarked']
df_dropped = df.drop(drop_columns, axis=1)

## Export Processed Data to Cleaned CSV

In [111]:
df_dropped.to_csv('train_cleaned.csv', index=False)